In [14]:
import keras
import pandas as pd
import numpy as np

In [15]:
from keras.layers import Conv1D,Dense
from keras.models import Sequential


In [16]:
x_train = np.load('x_train.npy')
x_test = np.load('x_test.npy')
test_val = np.load('test_val.npy')
y_test = np.load('y_test.npy')
y_train = np.load('y_train.npy')
y_val = np.load('y_val.npy')

In [9]:
y_train.shape

(37065, 20)

In [10]:

len(x_train[3])

1011

In [8]:
import tensorflow as tf
from keras.layers import Dense, Input, Embedding, Lambda, Dropout, Activation, SpatialDropout1D, Reshape, GlobalAveragePooling1D, merge, Flatten, Bidirectional, CuDNNGRU, add, Conv1D, GlobalMaxPooling1D
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras import optimizers
from keras import initializers
from keras.engine import InputSpec, Layer
from keras import backend as K

class AttentionWeightedAverage(Layer):

    def __init__(self, return_attention=False, **kwargs):
        self.init = initializers.get('uniform')
        self.supports_masking = True
        self.return_attention = return_attention
        super(AttentionWeightedAverage, self).__init__(** kwargs)

    def build(self, input_shape):
        self.input_spec = [InputSpec(ndim=3)]
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[2], 1),
                                 name='{}_W'.format(self.name),
                                 initializer=self.init)
        self.trainable_weights = [self.W]
        super(AttentionWeightedAverage, self).build(input_shape)

    def call(self, x, mask=None):
        # computes a probability distribution over the timesteps
        # uses 'max trick' for numerical stability
        # reshape is done to avoid issue with Tensorflow
        # and 1-dimensional weights
        logits = K.dot(x, self.W)
        x_shape = K.shape(x)
        logits = K.reshape(logits, (x_shape[0], x_shape[1]))
        ai = K.exp(logits - K.max(logits, axis=-1, keepdims=True))

        # masked timesteps have zero weight
        if mask is not None:
            mask = K.cast(mask, K.floatx())
            ai = ai * mask
        att_weights = ai / (K.sum(ai, axis=1, keepdims=True) + K.epsilon())
        weighted_input = x * K.expand_dims(att_weights)
        result = K.sum(weighted_input, axis=1)
        if self.return_attention:
            return [result, att_weights]
        return result

    def get_output_shape_for(self, input_shape):
        return self.compute_output_shape(input_shape)

    def compute_output_shape(self, input_shape):
        output_len = input_shape[2]
        if self.return_attention:
            return [(input_shape[0], output_len), (input_shape[0], input_shape[1])]
        return (input_shape[0], output_len)

    def compute_mask(self, input, input_mask=None):
        if isinstance(input_mask, list):
            return [None] * len(input_mask)
        else:
            return None

In [10]:
def Hamming_loss(y_true, y_pred):
    tmp = K.abs(y_true-y_pred)
    return K.mean(K.cast(K.greater(tmp,0.5),dtype=float))

In [4]:
from keras.engine import InputSpec, Layer
class squash_function(Layer):
    def squash(s, axis=-1, epsilon=1e-7, name=None):
        with tf.name_scope(name, default_name="squash"):
            squared_norm = tf.reduce_sum(tf.square(s), axis=axis,
                                         keep_dims=True)
            safe_norm = tf.sqrt(squared_norm + epsilon)
            squash_factor = squared_norm / (1. + squared_norm)
            unit_vector = s / safe_norm
            return squash_factor * unit_vector

In [14]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, GlobalMaxPooling1D, Dropout, Conv1D,GlobalAveragePooling1D,Bidirectional,GRU,LSTM,concatenate,Input
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from keras.losses import binary_crossentropy
from keras.optimizers import Adam
from keras_self_attention import SeqSelfAttention

filter_length = 128
num_classes = 20
max_words = 111396
maxlen = 1011

input1 = Input(shape=(maxlen,))
print(input1.shape)
 
x = Embedding(max_words, 20, input_length=maxlen)(input1)

x = squash_function()(x)

x = Conv1D(128, kernel_size=2, padding="valid", kernel_initializer="glorot_uniform")(x)

x = squash_function()(x)
print(x.shape)

attn1 = AttentionWeightedAverage()(x)
max_pool1 = GlobalMaxPooling1D()(x)
avg_pool1 = GlobalAveragePooling1D()(x)

conv1 = Conv1D(128, kernel_size=2, padding="valid", kernel_initializer="glorot_uniform")(x)
conv2 = Conv1D(128, kernel_size=2, padding="valid", kernel_initializer="glorot_uniform")(x)
conv3 = Conv1D(128, kernel_size=2, padding="valid", kernel_initializer="glorot_uniform")(x)
cat = concatenate([conv2,conv1])
cat2 = concatenate([cat,conv3])


conv_1 = Conv1D(128, kernel_size=2, padding="valid", kernel_initializer="glorot_uniform")(x)
conv_2 = Conv1D(128, kernel_size=2, padding="valid", kernel_initializer="glorot_uniform")(x)
conv_3 = Conv1D(128, kernel_size=2, padding="valid", kernel_initializer="glorot_uniform")(x)
cat_1 = concatenate([conv_2,conv_1])
cat_2 = concatenate([cat_1,conv_3])

conv4 = Conv1D(128, kernel_size=2, padding="valid", kernel_initializer="glorot_uniform")(x)
conv5 = Conv1D(128, kernel_size=2, padding="valid", kernel_initializer="glorot_uniform")(x)
conv6 = Conv1D(128, kernel_size=2, padding="valid", kernel_initializer="glorot_uniform")(x)
cat4 = concatenate([conv4,conv5])
cat5 = concatenate([cat,conv6])


conv_4 = Conv1D(128, kernel_size=2, padding="valid", kernel_initializer="glorot_uniform")(x)
conv_5 = Conv1D(128, kernel_size=2, padding="valid", kernel_initializer="glorot_uniform")(x)
conv_6 = Conv1D(128, kernel_size=2, padding="valid", kernel_initializer="glorot_uniform")(x)
cat_4 = concatenate([conv_4,conv_5])
cat_5 = concatenate([cat_4,conv_6])

m = concatenate([cat2,cat_2])
n = concatenate([cat_5,cat5])

x = concatenate([m,n])

x = squash_function()(x)

x = Conv1D(256,kernel_size=4,padding="valid",kernel_initializer="glorot_uniform")(x)
x = Conv1D(256,kernel_size=4,padding="valid",kernel_initializer="glorot_uniform")(x)
x = Conv1D(256,kernel_size=4,padding="valid",kernel_initializer="glorot_uniform")(x)
x = squash_function()(x)
x = Conv1D(256,kernel_size=4,padding="valid",kernel_initializer="glorot_uniform")(x)
x = Conv1D(256,kernel_size=4,padding="valid",kernel_initializer="glorot_uniform")(x)
x = Conv1D(256,kernel_size=4,padding="valid",kernel_initializer="glorot_uniform")(x)
x = squash_function()(x)


maxpool = GlobalMaxPooling1D()(x)
attn = AttentionWeightedAverage()(x)
avg = GlobalAveragePooling1D()(x)

l = concatenate([maxpool,avg])
s = concatenate([l,attn])
t = concatenate([s,attn1])
u = concatenate([avg_pool1,max_pool1])
x = concatenate([u,t])
x = squash_function()(x)
 
preds = Dense(num_classes, activation="sigmoid")(x)
 
model = keras.Model(input1, preds)
 
model.summary()
 

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[Hamming_loss])


# callbacks = [
#     ReduceLROnPlateau(), 
#     EarlyStopping(patience=4)
# ]

# history = model.fit(x_train, y_train,
#                     epochs=5,
#                     batch_size=128,
#                     validation_split=0.1,
#                     callbacks=callbacks)


(?, 1011)

(?, 1010, 128)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1011)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1011, 20)     2227920     input_1[0][0]                    
__________________________________________________________________________________________________
squash_function_1 (squash_funct (None, 1011, 20)     0           embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 1010, 128)    5248        squash_function_1[0][0]          
__________________________________________________________________________________


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [10]:
from sklearn.metrics import accuracy_score, roc_auc_score, log_loss

def _train_model(model, batch_size, train_x, train_y, test_val, y_val):
    num_labels = train_y.shape[1]
    patience = 7
    best_loss = -1
    best_weights = None
    best_epoch = 0
    
    current_epoch = 0
    
    while True:
        model.fit(x_train, y_train, batch_size=batch_size, epochs=1)
        y_pred = model.predict(test_val, batch_size=batch_size)

        total_loss = 0
        for j in range(num_labels):
            loss = log_loss(y_val[:, j], y_pred[:, j])
            total_loss += loss

        total_loss /= num_labels

        print("Epoch {0} loss {1} best_loss {2}".format(current_epoch, total_loss, best_loss))

        current_epoch += 1
        if total_loss < best_loss or best_loss == -1:
            best_loss = total_loss
            best_weights = model.get_weights()
            best_epoch = current_epoch
        else:
            if current_epoch - best_epoch == patience:
                break

    model.set_weights(best_weights)
    return model

In [11]:
batch_size = 32
history = _train_model(model,batch_size,x_train,y_train,test_val,y_val)


Epoch 1/1
37065/37065 [==============================] - 231s 6ms/step - loss: 0.4255 - Hamming_loss: 0.2082
Epoch 0 loss 0.4104676276572853 best_loss -1
Epoch 1/1
37065/37065 [==============================] - 226s 6ms/step - loss: 0.3943 - Hamming_loss: 0.1878
Epoch 1 loss 0.3951933538261482 best_loss 0.4104676276572853
Epoch 1/1
37065/37065 [==============================] - 225s 6ms/step - loss: 0.3800 - Hamming_loss: 0.1786
Epoch 2 loss 0.38703975597774976 best_loss 0.3951933538261482
Epoch 1/1
37065/37065 [==============================] - 226s 6ms/step - loss: 0.3698 - Hamming_loss: 0.1720
Epoch 3 loss 0.3837602303904269 best_loss 0.38703975597774976
Epoch 1/1
37065/37065 [==============================] - 225s 6ms/step - loss: 3.0981 - Hamming_loss: 0.2791


/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: divide by zero encoun

Epoch 4 loss nan best_loss 0.3837602303904269
Epoch 1/1
37065/37065 [==============================] - 224s 6ms/step - loss: 7.0926 - Hamming_loss: 0.4432


/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: divide by zero encoun

Epoch 5 loss nan best_loss 0.3837602303904269
Epoch 1/1
37065/37065 [==============================] - 224s 6ms/step - loss: 7.0926 - Hamming_loss: 0.4432


...

Epoch 8 loss nan best_loss 0.3837602303904269
Epoch 1/1
37065/37065 [==============================] - 224s 6ms/step - loss: 7.0926 - Hamming_loss: 0.4432


/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: divide by zero encoun

Epoch 9 loss nan best_loss 0.3837602303904269
Epoch 1/1
37065/37065 [==============================] - 224s 6ms/step - loss: 7.0926 - Hamming_loss: 0.4432


/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/metrics/classification.py:2174: RuntimeWarning: divide by zero encoun

Epoch 10 loss nan best_loss 0.3837602303904269


In [12]:
y_pred = history.predict(x_test, batch_size=batch_size)

In [13]:
attn1.shape

TensorShape([Dimension(None), Dimension(128)])

In [14]:
conv1.shape

TensorShape([Dimension(None), Dimension(1009), Dimension(128)])

In [15]:
conv2.shape

TensorShape([Dimension(None), Dimension(1009), Dimension(128)])

In [16]:
y_val

array([[0, 0, 0, ..., 0, 0, 1],
       [0, 0, 1, ..., 0, 0, 1],
       [0, 0, 0, ..., 1, 1, 1],
       ...,
       [0, 0, 0, ..., 1, 1, 0],
       [0, 0, 0, ..., 1, 1, 0],
       [0, 0, 1, ..., 0, 0, 1]])

In [17]:
y_test.shape

(8177, 20)

In [18]:
print(y_pred[35],y_val[35])

[0.00708026 0.00687671 0.03267449 0.01058167 0.03165734 0.05051699
 0.00675648 0.11864647 0.08914489 0.00560895 0.00180003 0.0550763
 0.01204166 0.33128622 0.97376716 0.06555697 0.00945166 0.00899646
 0.00953016 0.974888  ] [0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 0]


In [19]:
(7.8654468e-01+           
5.0963247e-01+
9.6992075e-01+
5.8361495e-01+
3.5057139e-01+
5.7766867e-01)/6

0.6296588183333333

In [20]:
y_pred_round = np.where(y_pred>0.5,1,0)#0.6 enough na lets see....happy ya

###### y_train.shape

In [21]:
from sklearn.metrics import jaccard_score,roc_auc_score,confusion_matrix,hamming_loss
print(jaccard_score(y_test,y_pred_round,average='micro'))
print(jaccard_score(y_test,y_pred_round,average='macro'))
print(roc_auc_score(y_test,y_pred_round))
print(hamming_loss(y_test,y_pred_round))

0.49901620271356956
0.3784311205774858
0.6671280591463095
0.17904488198605845


In [13]:
from keras.models import load_model
conv1 = load_model('conv1.h5',custom_objects={'squash_function': squash_function,
                                              'AttentionWeightedAverage':AttentionWeightedAverage,
                                              'Hamming_loss':Hamming_loss
                                             })
# model.save('conv3.h5') #nvm#done...
conv3 = load_model('conv3.h5',custom_objects={'squash_function': squash_function,
                                              'AttentionWeightedAverage':AttentionWeightedAverage,
                                              'Hamming_loss':Hamming_loss
                                             })

In [17]:
batch_size = 32

y_pred_conv3 = conv3.predict(x_test, batch_size=batch_size)

y_pred_conv1 = conv1.predict(x_test, batch_size=batch_size)


In [21]:
def corr(a, b):
    counts = 0
    for i in range(a.shape[0]):  
      # correlation
        print("Finding correlation row : {}".format(i))
      #print("Column to be measured: {}".format(prediction))
        cor = np.corrcoef(a[i,:],b[i,:])
        print("Pearson's correlation score: {}".format(cor))
        if(cor[0][1] < 0.9):  
            counts+=1
    return counts

In [22]:
l = [corr(y_pred_conv3, y_pred_conv1)] 

Finding correlation row : 0
Pearson's correlation score: [[1.         0.89155442]
 [0.89155442 1.        ]]
Finding correlation row : 1
Pearson's correlation score: [[1.         0.94018379]
 [0.94018379 1.        ]]
Finding correlation row : 2
Pearson's correlation score: [[1.         0.92659825]
 [0.92659825 1.        ]]
Finding correlation row : 3
Pearson's correlation score: [[1.         0.97189103]
 [0.97189103 1.        ]]
Finding correlation row : 4
Pearson's correlation score: [[1.         0.92027209]
 [0.92027209 1.        ]]
Finding correlation row : 5
Pearson's correlation score: [[1.        0.9700026]
 [0.9700026 1.       ]]
Finding correlation row : 6
Pearson's correlation score: [[1.         0.89039302]
 [0.89039302 1.        ]]
Finding correlation row : 7
Pearson's correlation score: [[1.         0.89901859]
 [0.89901859 1.        ]]
Finding correlation row : 8
Pearson's correlation score: [[1.         0.98597145]
 [0.98597145 1.        ]]
Finding correlation row : 9
Pear

Pearson's correlation score: [[1.         0.97435682]
 [0.97435682 1.        ]]
Finding correlation row : 235
Pearson's correlation score: [[1.         0.99355616]
 [0.99355616 1.        ]]
Finding correlation row : 236
Pearson's correlation score: [[1.         0.53437967]
 [0.53437967 1.        ]]
Finding correlation row : 237
Pearson's correlation score: [[1.        0.9925516]
 [0.9925516 1.       ]]
Finding correlation row : 238
Pearson's correlation score: [[1.        0.9149819]
 [0.9149819 1.       ]]
Finding correlation row : 239
Pearson's correlation score: [[1.         0.95306255]
 [0.95306255 1.        ]]
Finding correlation row : 240
Pearson's correlation score: [[1.         0.88797678]
 [0.88797678 1.        ]]
Finding correlation row : 241
Pearson's correlation score: [[1.         0.93946425]
 [0.93946425 1.        ]]
Finding correlation row : 242
Pearson's correlation score: [[1.         0.91391631]
 [0.91391631 1.        ]]
Finding correlation row : 243
Pearson's correlat

Pearson's correlation score: [[1.         0.93386757]
 [0.93386757 1.        ]]
Finding correlation row : 527
Pearson's correlation score: [[1.         0.93699224]
 [0.93699224 1.        ]]
Finding correlation row : 528
Pearson's correlation score: [[1.         0.93059084]
 [0.93059084 1.        ]]
Finding correlation row : 529
Pearson's correlation score: [[1.         0.91632808]
 [0.91632808 1.        ]]
Finding correlation row : 530
Pearson's correlation score: [[1.         0.88012105]
 [0.88012105 1.        ]]
Finding correlation row : 531
Pearson's correlation score: [[1.         0.91049079]
 [0.91049079 1.        ]]
Finding correlation row : 532
Pearson's correlation score: [[1.         0.98823143]
 [0.98823143 1.        ]]
Finding correlation row : 533
Pearson's correlation score: [[1.         0.96870622]
 [0.96870622 1.        ]]
Finding correlation row : 534
Pearson's correlation score: [[1.         0.87650416]
 [0.87650416 1.        ]]
Finding correlation row : 535
Pearson's 

Pearson's correlation score: [[1.         0.97338158]
 [0.97338158 1.        ]]
Finding correlation row : 827
Pearson's correlation score: [[1.         0.92043871]
 [0.92043871 1.        ]]
Finding correlation row : 828
Pearson's correlation score: [[1.         0.97705476]
 [0.97705476 1.        ]]
Finding correlation row : 829
Pearson's correlation score: [[1.         0.94584906]
 [0.94584906 1.        ]]
Finding correlation row : 830
Pearson's correlation score: [[1.         0.96340476]
 [0.96340476 1.        ]]
Finding correlation row : 831
Pearson's correlation score: [[1.         0.97550522]
 [0.97550522 1.        ]]
Finding correlation row : 832
Pearson's correlation score: [[1.         0.94619574]
 [0.94619574 1.        ]]
Finding correlation row : 833
Pearson's correlation score: [[1.         0.83512385]
 [0.83512385 1.        ]]
Finding correlation row : 834
Pearson's correlation score: [[1.         0.89532263]
 [0.89532263 1.        ]]
Finding correlation row : 835
Pearson's 

Pearson's correlation score: [[1.         0.92744831]
 [0.92744831 1.        ]]
Finding correlation row : 1105
Pearson's correlation score: [[1.         0.93447948]
 [0.93447948 1.        ]]
Finding correlation row : 1106
Pearson's correlation score: [[1.         0.88596866]
 [0.88596866 1.        ]]
Finding correlation row : 1107
Pearson's correlation score: [[1.         0.91449351]
 [0.91449351 1.        ]]
Finding correlation row : 1108
Pearson's correlation score: [[1.         0.96698017]
 [0.96698017 1.        ]]
Finding correlation row : 1109
Pearson's correlation score: [[1.        0.8719041]
 [0.8719041 1.       ]]
Finding correlation row : 1110
Pearson's correlation score: [[1.         0.80346155]
 [0.80346155 1.        ]]
Finding correlation row : 1111
Pearson's correlation score: [[1.         0.93683481]
 [0.93683481 1.        ]]
Finding correlation row : 1112
Pearson's correlation score: [[1.         0.97498962]
 [0.97498962 1.        ]]
Finding correlation row : 1113
Pears

...

Pearson's correlation score: [[1.         0.98009592]
 [0.98009592 1.        ]]
Finding correlation row : 6821
Pearson's correlation score: [[1.         0.97031541]
 [0.97031541 1.        ]]
Finding correlation row : 6822
Pearson's correlation score: [[1.        0.9460232]
 [0.9460232 1.       ]]
Finding correlation row : 6823
Pearson's correlation score: [[1.         0.92360255]
 [0.92360255 1.        ]]
Finding correlation row : 6824
Pearson's correlation score: [[1.         0.97524835]
 [0.97524835 1.        ]]
Finding correlation row : 6825
Pearson's correlation score: [[1.         0.97172262]
 [0.97172262 1.        ]]
Finding correlation row : 6826
Pearson's correlation score: [[1.        0.9710437]
 [0.9710437 1.       ]]
Finding correlation row : 6827
Pearson's correlation score: [[1.         0.93094613]
 [0.93094613 1.        ]]
Finding correlation row : 6828
Pearson's correlation score: [[1.        0.9129351]
 [0.9129351 1.       ]]
Finding correlation row : 6829
Pearson's cor

Pearson's correlation score: [[1.         0.90385266]
 [0.90385266 1.        ]]
Finding correlation row : 7116
Pearson's correlation score: [[1.         0.71607786]
 [0.71607786 1.        ]]
Finding correlation row : 7117
Pearson's correlation score: [[1.         0.96529657]
 [0.96529657 1.        ]]
Finding correlation row : 7118
Pearson's correlation score: [[1.        0.9552664]
 [0.9552664 1.       ]]
Finding correlation row : 7119
Pearson's correlation score: [[1.         0.70656325]
 [0.70656325 1.        ]]
Finding correlation row : 7120
Pearson's correlation score: [[1.         0.70957564]
 [0.70957564 1.        ]]
Finding correlation row : 7121
Pearson's correlation score: [[1.        0.9548362]
 [0.9548362 1.       ]]
Finding correlation row : 7122
Pearson's correlation score: [[1.         0.89218766]
 [0.89218766 1.        ]]
Finding correlation row : 7123
Pearson's correlation score: [[1.         0.54152453]
 [0.54152453 1.        ]]
Finding correlation row : 7124
Pearson's

Pearson's correlation score: [[1.         0.90863078]
 [0.90863078 1.        ]]
Finding correlation row : 7424
Pearson's correlation score: [[1.         0.93770582]
 [0.93770582 1.        ]]
Finding correlation row : 7425
Pearson's correlation score: [[1.         0.96264108]
 [0.96264108 1.        ]]
Finding correlation row : 7426
Pearson's correlation score: [[1.         0.91724842]
 [0.91724842 1.        ]]
Finding correlation row : 7427
Pearson's correlation score: [[1.         0.95125645]
 [0.95125645 1.        ]]
Finding correlation row : 7428
Pearson's correlation score: [[1.         0.92467207]
 [0.92467207 1.        ]]
Finding correlation row : 7429
Pearson's correlation score: [[1.         0.85925621]
 [0.85925621 1.        ]]
Finding correlation row : 7430
Pearson's correlation score: [[1.         0.98215044]
 [0.98215044 1.        ]]
Finding correlation row : 7431
Pearson's correlation score: [[1.         0.97110593]
 [0.97110593 1.        ]]
Finding correlation row : 7432
P

Pearson's correlation score: [[1.         0.93196285]
 [0.93196285 1.        ]]
Finding correlation row : 7745
Pearson's correlation score: [[1.         0.95039438]
 [0.95039438 1.        ]]
Finding correlation row : 7746
Pearson's correlation score: [[1.         0.94374132]
 [0.94374132 1.        ]]
Finding correlation row : 7747
Pearson's correlation score: [[1.         0.98790513]
 [0.98790513 1.        ]]
Finding correlation row : 7748
Pearson's correlation score: [[1.         0.88155305]
 [0.88155305 1.        ]]
Finding correlation row : 7749
Pearson's correlation score: [[1.         0.98473949]
 [0.98473949 1.        ]]
Finding correlation row : 7750
Pearson's correlation score: [[1.         0.97510805]
 [0.97510805 1.        ]]
Finding correlation row : 7751
Pearson's correlation score: [[1.         0.82620139]
 [0.82620139 1.        ]]
Finding correlation row : 7752
Pearson's correlation score: [[1.         0.91339772]
 [0.91339772 1.        ]]
Finding correlation row : 7753
P

Finding correlation row : 8054
Pearson's correlation score: [[1.         0.88886188]
 [0.88886188 1.        ]]
Finding correlation row : 8055
Pearson's correlation score: [[1.         0.63439065]
 [0.63439065 1.        ]]
Finding correlation row : 8056
Pearson's correlation score: [[1.         0.90106881]
 [0.90106881 1.        ]]
Finding correlation row : 8057
Pearson's correlation score: [[1.         0.82926972]
 [0.82926972 1.        ]]
Finding correlation row : 8058
Pearson's correlation score: [[1.        0.8348033]
 [0.8348033 1.       ]]
Finding correlation row : 8059
Pearson's correlation score: [[1.         0.90910577]
 [0.90910577 1.        ]]
Finding correlation row : 8060
Pearson's correlation score: [[1.        0.9542532]
 [0.9542532 1.       ]]
Finding correlation row : 8061
Pearson's correlation score: [[1.         0.91896988]
 [0.91896988 1.        ]]
Finding correlation row : 8062
Pearson's correlation score: [[1.         0.95674546]
 [0.95674546 1.        ]]
Finding c

In [23]:
l

[2072]

In [ ]:
#i think its better to check with those lgbm models yeah ok, or triansformer,capsnet